In [1]:
import pandas as pd
import numpy as np

In [2]:
from pyport.portopt import opt, objfunc

In [3]:
from pandas_datareader import data

In [4]:
"""Define parameters for data request."""

tickers = ['AGG', 'EBND', 'URTH', 'EEM']
data_source = 'yahoo'
start_date = '2014-03-31'
end_date = '2018-03-31'

In [5]:
"""Request data, ensure sorted by ascending date, and normalise."""

panel_data = data.DataReader(tickers, data_source, start_date, end_date)
px_data = panel_data.loc['Adj Close'].dropna().sort_index(ascending=True)
px_data = px_data / px_data.iloc[0]

In [6]:
"""
TEST CASE
Add constraints: Portfolio weights must sum to 100%,
Add boundaries: Long only, no short positions.
"""

cons = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
]

bnds = [(0, 1) for x in px_data.columns]

In [7]:
"""
Define the relaxation tolerance allowed for init_func.
For example, if prim_func is max_er and init_func is max_dr,
relax_tolderance=0.1 and num_of_steps=10 results in a tolerance of 10% from max_dr optimal in favour of max_er,
performed in 10 steps.
"""

relax_tolerance = 0.1
num_of_steps = 10

In [8]:
"""Run optimisation."""

opt_results = opt.dual_target_optimisation(
    objfunc.max_er, objfunc.max_dr, px_data, 
    relax_tol=relax_tolerance, steps=num_of_steps,
    rf=0.0, scaling_fact=252, 
    constraints=cons, bounds=bnds)

Total time: 3.9265191555023193 secs


In [9]:
"""View results."""

opt_results

,max_er,max_dr,AGG,EBND,EEM,URTH
step,,,,,,
0,0.029983,1.598375,0.7563,0.0821,0.0074,0.1542
1,0.032605,1.580615,0.7532,0.0402,0.0180,0.1886
2,0.033817,1.562856,0.7485,0.0235,0.0234,0.2046
3,0.034817,1.545096,0.7436,0.0105,0.0285,0.2174
4,0.035720,1.527336,0.7378,0.0000,0.0330,0.2292
5,0.029108,1.509576,0.8726,0.0000,0.0000,0.1274
6,0.037344,1.491817,0.7059,0.0000,0.0356,0.2584
7,0.038111,1.474056,0.6908,0.0000,0.0376,0.2717
8,0.038869,1.456297,0.6757,0.0000,0.0399,0.2844
